## Imports

In [1]:
import tensorflow as tf

## Result classifier

In [4]:
result_classifier = tf.keras.models.load_model("model_artifacts/result_classifier/saved_model/chessformer_result_classifier.keras")
result_classifier.summary()

Model: "chessformer_result_classifier_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_1 (Encoder)         multiple                  7395840   
                                                                 
 result_classification_1 (R  multiple                  37187     
 esultClassification)                                            
                                                                 
Total params: 7433027 (28.35 MB)
Trainable params: 7433027 (28.35 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
